In [1]:
!pip install owslib rasterio requests fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 47.5 MB/s eta 0:00:00


In [2]:
from owslib.wcs import WebCoverageService
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
import rasterio.warp

# URL WCS сервиса
wcs_url = "https://maps.isric.org/mapserv?map=/map/ocs.map"

# Подключаемся к WCS
wcs = WebCoverageService(wcs_url, version='2.0.1')

# Выбираем покрытие
cov_id = 'ocs_0-30cm_mean'
ph_0_5 = wcs.contents[cov_id]

# Определяем систему координат и границы
subsets = [('X', 4000000, 8000000), ('Y', 4100000, 6100000)] #(new_x_min, new_x_max), (new_y_min, new_y_max) - Казахстан
crs = "http://www.opengis.net/def/crs/EPSG/0/152160"

# Запрашиваем покрытие
response = wcs.getCoverage(
    identifier=[cov_id],
    crs=crs,
    subsets=subsets,
    resx=250, resy=250,
    format=ph_0_5.supportedFormats[0]
)

# Сохраняем TIFF-файл
raw_filename = "Kazakhstan_ocs_0-30cm_mean_raw.tif"
with open(raw_filename, 'wb') as file:
    file.write(response.read())

print(f"Файл сохранен: {raw_filename}")

# Преобразование в EPSG:4326
output_filename = "Kazakhstan_ocs_0-30cm_mean_prj.tif"
with rasterio.open(raw_filename) as src:
    transform, width, height = rasterio.warp.calculate_default_transform(
        src.crs, "EPSG:4326", src.width, src.height, *src.bounds)

    kwargs = src.meta.copy()
    kwargs.update({
        'crs': "EPSG:4326",
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            rasterio.warp.reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs="EPSG:4326",
                resampling=rasterio.enums.Resampling.nearest
            )

print(f"Файл преобразован и сохранен в {output_filename}")

Файл сохранен: Kazakhstan_ocs_0-30cm_mean_raw.tif
Файл преобразован и сохранен в Kazakhstan_ocs_0-30cm_mean_prj.tif
